## Introduction

## 4.1 Encoding Nominal Categorical Features

You have a feature with nominal classes that has no intrinsic ordering (e.g., apple,
pear, banana).

One-hot encode the feature using scikit-learn’s LabelBinarizer

In [18]:
# Import libraries
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

In [19]:
# Create feature
feature = np.array([["Texas"],
                    ["California"],
                    ["Texas"],
                    ["Delaware"],
                    ["Texas"]])

In [20]:
# Create one-hot encoder
one_hot = LabelBinarizer()
# One-hot encode feature
one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

We can use the **classes_ method** to output the classes

In [21]:
# View feature classes
one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

If we want to reverse the one-hot encoding, we can use **inverse_transform**

In [22]:
# Reverse one-hot encoding
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

We can even use pandas to one-hot encode the feature

In [23]:
# Import library
import pandas as pd
# Create dummy variables from feature
pd.get_dummies(feature[:,0])

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


One helpful ability of scikit-learn is to handle a situation where each observation lists
multiple classes:

In [24]:
# Create multiclass feature
multiclass_feature = [("Texas", "Florida"),
                    ("California", "Alabama"),
                    ("Texas", "Florida"),
                   ("Delware", "Florida"),
                   ("Texas", "Alabama")]
# Create multiclass one-hot encoder
one_hot_multiclass = MultiLabelBinarizer()
# One-hot encode multiclass feature
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

Once again, we can see the classes with the **classes_ method**:

In [25]:
# View classes
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delware', 'Florida', 'Texas'],
      dtype=object)

## 4.2 Encoding Ordinal Categorical Features

Use pandas DataFrame’s replace method to transform string labels to numerical
equivalents:

In [26]:
# Load library
import pandas as pd
# Create features
dataframe = pd.DataFrame({"Score": ["Low", "Low", "Medium", "Medium", "High"]})
# Create mapper
scale_mapper = {"Low":1,
                "Medium":2,
                "High":3}
# Replace feature values with scale
dataframe["Score"].replace(scale_mapper)


0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

In [28]:
dataframe = pd.DataFrame({"Score": ["Low", "Low", "Medium", "Medium", "High"]})
dataframe

,Score
0,Low
1,Low
2,Medium
3,Medium
4,High


In [29]:
dataframe = pd.DataFrame({"Score": ["Low",
                                    "Low",
                                    "Medium",
                                    "Medium",
                                    "High",
                                    "Barely More Than Medium"]})
scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium": 3,
                "High":4}
dataframe["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    4
5    3
Name: Score, dtype: int64

In [32]:
dataframe = pd.DataFrame({"Score": ["Low",
                                    "Low",
                                    "Medium",
                                    "Medium",
                                    "High",
                                    "Barely More Than Medium"]})
dataframe

,Score
0,Low
1,Low
2,Medium
3,Medium
4,High
5,Barely More Than Medium


In this example, the distance between **Low** and **Medium** is the same as the distance
between Medium and Barely More Than Medium, which is almost certainly not accurate.
The best approach is to be conscious about the numerical values mapped to
classes:

In [33]:
scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium": 2.1,
                "High":3}
dataframe["Score"].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
5    2.1
Name: Score, dtype: float64

## 4.3 Encoding Dictionaries of Features

In [34]:
# Import library
from sklearn.feature_extraction import DictVectorizer
# Create dictionary
data_dict = [{"Red": 2, "Blue": 4},
            {"Red": 4, "Blue": 3},
            {"Red": 1, "Yellow": 2},
            {"Red": 2, "Yellow": 2}]
# Create dictionary vectorizer
dictvectorizer = DictVectorizer(sparse=False)
# Convert dictionary to feature matrix
features = dictvectorizer.fit_transform(data_dict)
# View feature feature matrix
features

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

By default **DictVectorizer** outputs a sparse matrix that only stores elements with a
value other than 0. This can be very helpful when we have massive matrices (often
encountered in natural language processing) and want to minimize the memory
requirements. We can force **DictVectorizer** to output a dense matrix using
**sparse=False.**
We can get the names of each generated feature using the **get_feature_names**
method:

In [35]:
# Get feature names
feature_names = dictvectorizer.get_feature_names()
# View feature names
feature_names

['Blue', 'Red', 'Yellow']

While not necessary, for the sake of illustration we can create a pandas DataFrame to
view the output better:

In [36]:
# Import library
import pandas as pd
# Create dataframe from features
pd.DataFrame(features, columns=feature_names)

,Blue,Red,Yellow
0,4.0,2.0,0.0
1,3.0,4.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


In [37]:
# Create word counts dictionaries for four documents
doc_1_word_count = {"Red": 2, "Blue": 4}
doc_2_word_count = {"Red": 4, "Blue": 3}
doc_3_word_count = {"Red": 1, "Yellow": 2}
doc_4_word_count = {"Red": 2, "Yellow": 2}
# Create list
doc_word_counts = [doc_1_word_count,
                   doc_2_word_count,
                   doc_3_word_count,
                   doc_4_word_count]
# Convert list of word count dictionaries into feature matrix
dictvectorizer.fit_transform(doc_word_counts)

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

In our toy example there are only three unique words (Red, Yellow, Blue) so there are
only three features in our matrix; however, you can imagine that if each document
was actually a book in a university library our feature matrix would be very large (and
then we would want to set spare to True).

## 4.4 Imputing Missing Class Values

In [38]:
# Load libraries
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
# Create feature matrix with categorical feature
X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])
# Create feature matrix with missing values in the categorical feature
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                      [np.nan, -0.67, -0.22]])
# Train KNN learner
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:,1:], X[:,0])
# Predict missing values' class
imputed_values = trained_model.predict(X_with_nan[:,1:])


In [40]:
imputed_values.reshape(-1,1)

array([[0.],
       [1.]])

In [47]:
# Join column of predicted class with their other features
X_with_imputed = np.hstack((imputed_values.reshape(-1,1), X_with_nan[:,1:]))


In [48]:
X_with_imputed

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22]])

In [49]:
# Join two feature matrices
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

### difference between hstack and  vstack ###

An alternative solution is to fill in missing values with the feature’s most frequent
value:

In [41]:
a = np.ones((3, 3))
np.vstack( (a, np.array((2,2,2))) )

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [2., 2., 2.]])

Adding a column requires a bit more work, however. You can't use **np.hstack** directly:

In [ ]:
 a = np.ones((3, 3))
np.hstack( (a, np.array((2,2,2))) )

This is because **np.hstack** cannot concatenate two arrays with different numbers of rows. Schematically:

We can't simply transpose our new row, either, because it's a one-dimensional array and its transpose is the same shape as the original. So we need to reshape it first:

In [44]:
a = np.ones((3, 3))
b = np.array((2,2,2)).reshape(3,1)

In [45]:
b

array([[2],
       [2],
       [2]])

In [46]:
np.hstack((a, b))

array([[1., 1., 1., 2.],
       [1., 1., 1., 2.],
       [1., 1., 1., 2.]])

An alternative solution is to fill in missing values with the feature’s most frequent
value:

In [15]:
from sklearn.impute import SimpleImputer

In [50]:
# Join the two feature matrices
X_complete = np.vstack((X_with_nan, X))
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

## 4.5 Handling Imbalanced Classes

You have a target vector with highly imbalanced classes.

Collect more data. If that isn’t possible, change the metrics used to evaluate your
model. If that doesn’t work, consider using a model’s built-in class weight parameters
(if available), downsampling, or upsampling. We cover evaluation metrics in a later
chapter, so for now let us focus on class weight parameters, downsampling, and
upsampling.
To demonstrate our solutions, we need to create some data with imbalanced classes.
Fisher’s Iris dataset contains three balanced classes of 50 observations, each indicating
the species of flower *(Iris setosa, Iris virginica, and Iris versicolor)*. To unbalance the
dataset, we remove 40 of the 50 Iris setosa observations and then merge the *Iris virginica*
and *Iris versicolor* classes. The end result is a binary target vector indicating if anobservation is an Iris setosa flower or not. The result is 10 observations of *Iris setosa*
(class 0) and 100 observations of not Iris setosa (class 1):

In [51]:
# Load libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
# Load iris data
iris = load_iris()
# Create feature matrix
features = iris.data
# Create target vector
target = iris.target


In [53]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [54]:
# Remove first 40 observations
features = features[40:,:]
target = target[40:]
# Create binary target vector indicating if class 0
target = np.where((target == 0), 0, 1)
# Look at the imbalanced target vector
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Many algorithms in scikit-learn offer a parameter to weight classes during training to
counteract the effect of their imbalance. While we have not covered it yet, **RandomFor
estClassifier** is a popular classification algorithm and includes a **class_weight**
parameter. You can pass an argument specifying the desired class weights explicitly:

In [55]:
# Create weights
weights = {0: .9, 1: 0.1}
# Create random forest classifier with weights
RandomForestClassifier(class_weight=weights)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 0.9, 1: 0.1}, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

Or you can pass balanced, which automatically creates weights inversely proportional
to class frequencies:

In [56]:
# Train a random forest with balanced class weights
RandomForestClassifier(class_weight="balanced")

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Alternatively, we can downsample the majority class or upsample the minority class.
In downsampling, we randomly sample without replacement from the majority class
(i.e., the class with more observations) to create a new subset of observations equal in
size to the minority class. For example, if the minority class has 10 observations, we
will randomly select 10 observations from the majority class and use those 20 observations
as our data. Here we do exactly that using our unbalanced Iris data:

In [57]:
# Indicies of each class' observations
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]
# Number of observations in each class
n_class0 = len(i_class0)
n_class1 = len(i_class1)

In [60]:
i_class0

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [61]:
# For every observation of class 0, randomly sample
# from class 1 without replacement
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

In [62]:
i_class1_downsampled 

array([ 23,  26,  57,  93,  84, 104,  53, 101,  69,  61], dtype=int64)

In [63]:
# Join together class 0's target vector with the
# downsampled class 1's target vector
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [65]:
# Join together class 0's feature matrix with the
# downsampled class 1's feature matrix
np.vstack((features[i_class0,:], features[i_class1_downsampled,:]))

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4],
       [4.8, 3. , 1.4, 0.3],
       [5.1, 3.8, 1.6, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5. , 3.3, 1.4, 0.2],
       [6.1, 2.9, 4.7, 1.4],
       [5.6, 3. , 4.5, 1.5],
       [6.2, 2.9, 4.3, 1.3],
       [6.3, 2.8, 5.1, 1.5],
       [6.7, 3.3, 5.7, 2.1],
       [6.7, 3.3, 5.7, 2.5],
       [5. , 2.3, 3.3, 1. ],
       [6.9, 3.1, 5.1, 2.3],
       [7.2, 3.6, 6.1, 2.5],
       [5.8, 2.7, 5.1, 1.9]])

Our other option is to upsample the minority class. In upsampling, for every observation
in the majority class, we randomly select an observation from the minority class
with replacement. The end result is the same number of observations from the minority and majority classes. Upsampling is implemented very similarly to downsampling,
just in reverse:

In [ ]:
# For every observation in class 1, randomly sample from class 0 with replacement
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

In [ ]:
# Join together class 0's upsampled target vector with class 1's target vector
np.concatenate((target[i_class0_upsampled], target[i_class1]))

In [ ]:
# Join together class 0's upsampled feature matrix with class 1's feature matrix
np.vstack((features[i_class0_upsampled,:], features[i_class1,:]))[0:5]

## Explanation

In the real world, imbalanced classes are everywhere—most visitors don’t click the
buy button and many types of cancer are thankfully rare. For this reason, handling
imbalanced classes is a common activity in machine learning.
Our best strategy is simply to collect more observations—especially observations
from the minority class. However, this is often just not possible, so we have to resort
to other options.
A second strategy is to use a model evaluation metric better suited to imbalanced
classes. Accuracy is often used as a metric for evaluating the performance of a model,
but when imbalanced classes are present accuracy can be ill suited. For example, if
only 0.5% of observations have some rare cancer, then even a naive model that
predicts nobody has cancer will be 99.5% accurate. Clearly this is not ideal. Some better
metrics we discuss in later chapters are confusion matrices, precision, recall, F1
scores, and ROC curves.
A third strategy is to use the class weighing parameters included in implementations
of some models. This allows us to have the algorithm adjust for imbalanced classes.
Fortunately, many scikit-learn classifiers have a **class_weight** parameter, making it a
good option.The fourth and fifth strategies are related: downsampling and upsampling. In downsampling
we create a random subset of the majority class of equal size to the minority
class. In upsampling we repeatedly sample with replacement from the minority class
to make it of equal size as the majority class. The decision between using downsampling
and upsampling is context-specific, and in general we should try both to see
which produces better results.